# LSTM

In [1]:
import torch
import torch.nn as nn
from torchtext.datasets import IMDB

print("GPU:", torch.backends.mps.is_available())
print("GPU:", torch.cuda.is_available())
device = torch.device("cpu")
torch.manual_seed(123)


GPU: False
GPU: False


In [2]:
# TEXT = data.Field(tokenize="spacy")
# LABEL = data.LabelField(dtype=torch.float)
# train_data, test_data = IMDB.splits(TEXT, LABEL)
# 
# print("len of train data:", len(train_data))
# print("len of test data:", len(test_data))
# 
# print(train_data.examples[15].text)
# print(train_data.examples[15].label)
# 
# # word2vec, glove
# TEXT.build_vocab(train_data, max_size=10000, vectors="glove.6B.100d")
# LABEL.build_vocab(train_data)


需要注意的是, 在新版当中, tokenlizer的TEXT和FIELD已经移除了, 可以使用utils中的get_tokenizer和vacob包来进行分词处理. 而输出的结果label也变成了1和2. 原本的输出应该是pos,neg

In [4]:
from torchtext.data.utils import get_tokenizer 
from torchtext.vocab import build_vocab_from_iterator 

train_data_iter, test_data_iter = IMDB(split=('train','test'))
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter): 
    for _, text in data_iter: 
        yield tokenizer(text) 

def get_vocab(train_data_pipe): 
    vocab = build_vocab_from_iterator(yield_tokens(train_data_pipe), 
                                      specials=['<UNK>', '<PAD>']) 
    vocab.set_default_index(vocab['<UNK>']) 
    return vocab 

# 在新版当中我们使用Vocab包对数据进行编码, 获得所有的词汇量
# build_vocab_from_iterator：根据给定的迭代器yield_tokens(train_iter)，来构造一个Vocab对象。具体的Vocab类的介绍Vocab类。
# 得到一个库,就是将文本转为编码
train_vocab = get_vocab(train_data_iter)

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: torchtext._torchtext.Vocab, arg0: str) -> int

Invoked with: <torchtext._torchtext.Vocab object at 0x000002C41757CF70>, 0

In [5]:
print(len(train_vocab))
sentence = "i have a apple"
sentence_token = tokenizer(sentence)
print(sentence_token)
print(train_vocab(sentence_token))

100684
['i', 'have', 'a', 'apple']
[13, 33, 6, 7316]
